In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()],)

tail_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a fairy tail expert. You tell about the fairy tail's story that human ask."),
    ("human", "{fairy_tail}")
])

tail_chain = tail_prompt | chat

TypeError: 'method' object is not subscriptable

In [ ]:
odd_story_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a very good story teller. You get a story, and You answer about it but twist the plot little oddly."),
    ("human", "{story}")
])

odd_chain = odd_story_prompt | chat

In [ ]:
all_chain = {"story": tail_chain} | odd_chain
all_chain.invoke({
    "fairy_tail": "Peter Pan"
})